In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

def parse_hmm(fname):
    f = open(fname)
    line=f.readline()
    while line[0]!='#':
        line=f.readline()
    f.readline()
    f.readline()
    f.readline()
    f.readline()
    seq = []
    extras = np.zeros([0,10])
    prob = np.zeros([0,20])
    line = f.readline()
    while line[0:2]!='//':
        lineinfo = line.split()
        seq.append(lineinfo[0])
        probs_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(2,22)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)

        line = f.readline()
        lineinfo = line.split()
        extras_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(0,10)]
        extras = np.concatenate((extras,np.matrix(extras_)),axis=0)

        line = f.readline()
        assert len(line.strip())==0

        line = f.readline()
    return (''.join(seq),prob,extras)

## HMM Profile Extraction

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/HDA/Novel_BinaryML.csv')

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
hmm_matrices = []

for i in df['PDBid']:
    hmm_mat = parse_hmm(f'/content/drive/MyDrive/HDA/Novel_HMM/{i}.txt')
    hmm_arr = np.array(hmm_mat[1])
    hmm_matrices.append(hmm_arr)

# Add the list of hmm_arr as a new column 'hmm_matrix' in the DataFrame
df['hmm_matrix'] = hmm_matrices

## Skipxgram Extraction

In [ ]:
import numpy as np

def calculate_sxgbg_features(evolutionary_profile, X):

    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1,L - X):
                sxgbg_value += evolutionary_profile[l-1, i] * evolutionary_profile[l + X , j]

            sxgbg_matrix[i, j] = sxgbg_value

    return sxgbg_matrix

### S0G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s0g_matrix = []

for i in df['hmm_matrix']:
    s0g_mat = calculate_sxgbg_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_matrix.append(s0g_arr)

df['s0g_matrix'] = s0g_matrix

### S1G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s1g_matrix = []

for i in df['hmm_matrix']:
    s1g_mat = calculate_sxgbg_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_matrix.append(s1g_arr)

df['s1g_matrix'] = s1g_matrix

### S2G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s2g_matrix = []

for i in df['hmm_matrix']:
    s2g_mat = calculate_sxgbg_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_matrix.append(s2g_arr)

df['s2g_matrix'] = s2g_matrix

### S3G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s3g_matrix = []

for i in df['hmm_matrix']:
    s3g_mat = calculate_sxgbg_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_matrix.append(s3g_arr)

df['s3g_matrix'] = s3g_matrix

### S4G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s4g_matrix = []

for i in df['hmm_matrix']:
    s4g_mat = calculate_sxgbg_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_matrix.append(s4g_arr)

df['s4g_matrix'] = s4g_matrix

### S5G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s5g_matrix = []

for i in df['hmm_matrix']:
    s5g_mat = calculate_sxgbg_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_matrix.append(s5g_arr)

df['s5g_matrix'] = s5g_matrix

### S6G Matrix

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s6g_matrix = []

for i in df['hmm_matrix']:
    s6g_mat = calculate_sxgbg_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_matrix.append(s6g_arr)

df['s6g_matrix'] = s6g_matrix

## Evaluation Metrics

In [ ]:
def calculate_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    # Initialize accuracy
    accuracy = 0

    # Calculate accuracy for each instance
    for i in range(len(y_true)):
        # Calculate intersection and union
        intersection = np.sum(np.logical_and(y_true[i], y_pred[i]))
        union = np.sum(np.logical_or(y_true[i], y_pred[i]))

        # Add to total accuracy
        accuracy += intersection / union

    # Calculate average accuracy
    accuracy /= len(y_true)

    return accuracy

In [ ]:
import numpy as np

def norm_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    acc = []
    # Loop over each instance
    for i in range(len(y_true)):
        # Calculate the number of correct predictions for this instance
        print(y_true[i], y_pred[i])
        correct_predictions = np.sum(y_true[i] == y_pred[i])
        print(correct_predictions/5)
        acc.append(correct_predictions/5)

    # Calculate accuracy
    accuracy = sum(acc) / len(y_true)

    return accuracy

## S0G Matrix Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Define the model
model_s0g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s0g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 3s 24ms/step - loss: 0.4707 - accuracy: 0.4271 - val_loss: 0.5267 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 1s 15ms/step - loss: 0.4287 - accuracy: 0.4792 - val_loss: 0.4122 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 13ms/step - loss: 0.4105 - accuracy: 0.4896 - val_loss: 0.4620 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3924 - accuracy: 0.4792 - val_loss: 0.3560 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 8ms/step - loss: 0.3496 - accuracy: 0.5000 - val_loss: 0.3554 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3622 - accuracy: 0.5208 - val_loss: 0.3300 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 1s 11ms/step - loss: 0.34

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 4s 23ms/step - loss: 0.3416 - accuracy: 0.5876 - val_loss: 0.2770 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 15ms/step - loss: 0.3053 - accuracy: 0.6701 - val_loss: 0.2499 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 16ms/step - loss: 0.3003 - accuracy: 0.6392 - val_loss: 0.2258 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 15ms/step - loss: 0.3189 - accuracy: 0.6598 - val_loss: 0.2616 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
46/49 [===========================>..] - ETA: 0s - loss: 0.2961 - accuracy: 0.6848
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 17ms/step - loss: 0.2983 - accuracy: 0.6804 - val_loss: 0.2296 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 14ms/step - loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 15ms/step - loss: 0.3141 - accuracy: 0.6804 - val_loss: 0.2645 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.3031 - accuracy: 0.7010 - val_loss: 0.2573 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 10ms/step - loss: 0.2787 - accuracy: 0.6701 - val_loss: 0.2731 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2575 - accuracy: 0.7732 - val_loss: 0.2227 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 10ms/step - loss: 0.2374 - accuracy: 0.7835 - val_loss: 0.2392 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2450 - accuracy: 0.7629 - val_loss: 0.2486 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2609 - accur

...

49/49 [==============================] - 2s 12ms/step - loss: 0.2425 - accuracy: 0.7320 - val_loss: 0.2302 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 9ms/step - loss: 0.2124 - accuracy: 0.7526 - val_loss: 0.1900 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 10ms/step - loss: 0.2091 - accuracy: 0.7835 - val_loss: 0.1945 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1849 - accuracy: 0.7835 - val_loss: 0.2381 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
46/49 [===========================>..] - ETA: 0s - loss: 0.1996 - accuracy: 0.7717
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 10ms/step - loss: 0.1956 - accuracy: 0.7835 - val_loss: 0.2131 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1614 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 9ms/step - loss: 0.2240 - accuracy: 0.7835 - val_loss: 0.2468 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1660 - accuracy: 0.8557 - val_loss: 0.3356 - val_accuracy: 0.5417 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.1753 - accuracy: 0.8144 - val_loss: 0.2231 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 12ms/step - loss: 0.1703 - accuracy: 0.8660 - val_loss: 0.2046 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 13ms/step - loss: 0.1594 - accuracy: 0.8557 - val_loss: 0.2443 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 13ms/step - loss: 0.1997 - accuracy: 0.8144 - val_loss: 0.2319 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 7/15
46/49 [===========================>..] - ETA: 0s - loss: 0.1352 - accuracy: 0.9

1/1 [==============================] - 0s 116ms/step
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 1] [0 0 0 1 0]
0.8
[0 1 0 0 0] [0 0 0 0 1]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 1] [0 0 0 1 0]
0.4
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 1 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
Average Accuracy:  0.6203333333333333
Accuracy:  0.8961333333333336
Average Normalized Accuracy:  0.6495
Average Precision:  0.8134618714618714
Average Recall:  0.6334285714285715
F1 score: 0.7122446536067434
Grand Mean: 0.7208502938606421


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S1G Matrix Model

In [ ]:
# Define the model
model_s1g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s1g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 20ms/step - loss: 0.5539 - accuracy: 0.4375 - val_loss: 0.4590 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 10ms/step - loss: 0.4466 - accuracy: 0.5208 - val_loss: 0.4130 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 11ms/step - loss: 0.4103 - accuracy: 0.5208 - val_loss: 0.3687 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 10ms/step - loss: 0.3844 - accuracy: 0.5312 - val_loss: 0.3407 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 5/15
46/48 [===========================>..] - ETA: 0s - loss: 0.3677 - accuracy: 0.5109
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 1s 11ms/step - loss: 0.3693 - accuracy: 0.5000 - val_loss: 0.4165 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 10ms/step - loss

1/1 [==============================] - 0s 117ms/step
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [0 0 0 1 0]
0.6
[0 0 0 1 0] [1 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 1 0 0 0] [1 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 1 0]
0.8
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 1 0]
0.6
[0 1 0 0 0] [0 0 0 0 0]
0.8
Epoch 1/15


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 16ms/step - loss: 0.3764 - accuracy: 0.6392 - val_loss: 0.2855 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 9ms/step - loss: 0.3351 - accuracy: 0.6701 - val_loss: 0.2608 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.3003 - accuracy: 0.6701 - val_loss: 0.2534 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 8ms/step - loss: 0.3195 - accuracy: 0.7113 - val_loss: 0.2892 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 8ms/step - loss: 0.3171 - accuracy: 0.7320 - val_loss: 0.2432 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2892 - accuracy: 0.6701 - val_loss: 0.2450 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 0s 10ms/step - loss: 0.2685 - accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 16ms/step - loss: 0.2814 - accuracy: 0.7216 - val_loss: 0.2993 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2921 - accuracy: 0.7113 - val_loss: 0.2759 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2678 - accuracy: 0.7010 - val_loss: 0.2877 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 7ms/step - loss: 0.2341 - accuracy: 0.7216 - val_loss: 0.2089 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.2338 - accuracy: 0.7320 - val_loss: 0.2690 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2094 - accuracy: 0.7732 - val_loss: 0.2339 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 7/15
44/49 [=========================>....] - ETA: 0s - loss: 0.2545 - accuracy: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 10ms/step - loss: 0.1860 - accuracy: 0.7938 - val_loss: 0.2442 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1773 - accuracy: 0.8144 - val_loss: 0.3627 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2570 - accuracy: 0.7629 - val_loss: 0.2693 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
41/49 [========================>.....] - ETA: 0s - loss: 0.1857 - accuracy: 0.8293
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.2053 - accuracy: 0.7938 - val_loss: 0.2543 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 8ms/step - loss: 0.1699 - accuracy: 0.8144 - val_loss: 0.2531 - val_accuracy: 0.6667 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 8ms/step - loss: 0.1457 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S2G Matrix Model

In [ ]:
# Define the model
model_s2g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s2g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 15ms/step - loss: 0.5248 - accuracy: 0.4375 - val_loss: 0.4020 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 1s 11ms/step - loss: 0.4367 - accuracy: 0.4479 - val_loss: 0.3951 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 12ms/step - loss: 0.4073 - accuracy: 0.5104 - val_loss: 0.3838 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 16ms/step - loss: 0.3555 - accuracy: 0.6146 - val_loss: 0.4479 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 1s 13ms/step - loss: 0.4870 - accuracy: 0.5208 - val_loss: 0.4053 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 6/15
45/48 [===========================>..] - ETA: 0s - loss: 0.3627 - accuracy: 0.6111
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 1s 16ms/step - loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 18ms/step - loss: 0.3454 - accuracy: 0.6392 - val_loss: 0.2817 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 10ms/step - loss: 0.3327 - accuracy: 0.6907 - val_loss: 0.4914 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.3536 - accuracy: 0.6289 - val_loss: 0.2440 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 8ms/step - loss: 0.3046 - accuracy: 0.6804 - val_loss: 0.2333 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.2891 - accuracy: 0.8041 - val_loss: 0.3200 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2708 - accuracy: 0.7526 - val_loss: 0.2134 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 7/15
46/49 [===========================>..] - ETA: 0s - loss: 0.2341 - accuracy: 0.7

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 23ms/step - loss: 0.2994 - accuracy: 0.7629 - val_loss: 0.2022 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 17ms/step - loss: 0.2520 - accuracy: 0.7732 - val_loss: 0.2667 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 16ms/step - loss: 0.2547 - accuracy: 0.7938 - val_loss: 0.1953 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
47/49 [===========================>..] - ETA: 0s - loss: 0.2576 - accuracy: 0.6809
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 13ms/step - loss: 0.2551 - accuracy: 0.6907 - val_loss: 0.3301 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2586 - accuracy: 0.7629 - val_loss: 0.1725 - val_accuracy: 0.7917 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 13ms/step - loss: 0.205

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 19ms/step - loss: 0.2909 - accuracy: 0.6907 - val_loss: 0.2406 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2122 - accuracy: 0.8454 - val_loss: 0.2166 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2432 - accuracy: 0.7320 - val_loss: 0.2201 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
44/49 [=========================>....] - ETA: 0s - loss: 0.2243 - accuracy: 0.7614
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.2232 - accuracy: 0.7732 - val_loss: 0.3178 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1862 - accuracy: 0.8247 - val_loss: 0.2092 - val_accuracy: 0.7083 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 15ms/step - loss: 0.163

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 5s 25ms/step - loss: 0.2953 - accuracy: 0.7423 - val_loss: 0.2796 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 13ms/step - loss: 0.2193 - accuracy: 0.8144 - val_loss: 0.4555 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 17ms/step - loss: 0.2134 - accuracy: 0.8041 - val_loss: 0.2580 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2211 - accuracy: 0.7835 - val_loss: 0.3079 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.2388 - accuracy: 0.7732 - val_loss: 0.4137 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 6/15
44/49 [=========================>....] - ETA: 0s - loss: 0.2038 - accuracy: 0.8068
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 12ms/step - loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S3G Matrix Model

In [ ]:
# Define the model
model_s3g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s3g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 3s 20ms/step - loss: 0.5133 - accuracy: 0.4062 - val_loss: 0.3679 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 1s 12ms/step - loss: 0.4089 - accuracy: 0.5104 - val_loss: 0.3677 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3858 - accuracy: 0.5833 - val_loss: 0.4763 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 11ms/step - loss: 0.3907 - accuracy: 0.6562 - val_loss: 0.3654 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 1s 14ms/step - loss: 0.3564 - accuracy: 0.6667 - val_loss: 0.3118 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3453 - accuracy: 0.6146 - val_loss: 0.3195 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 10ms/step - loss: 0.2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 19ms/step - loss: 0.3412 - accuracy: 0.7010 - val_loss: 0.1975 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2976 - accuracy: 0.6598 - val_loss: 0.2140 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2931 - accuracy: 0.7423 - val_loss: 0.2317 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 7ms/step - loss: 0.3053 - accuracy: 0.7320 - val_loss: 0.2887 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
47/49 [===========================>..] - ETA: 0s - loss: 0.2602 - accuracy: 0.7553
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 8ms/step - loss: 0.2597 - accuracy: 0.7526 - val_loss: 0.2773 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.2127 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.3209 - accuracy: 0.7526 - val_loss: 0.2385 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2681 - accuracy: 0.7835 - val_loss: 0.3251 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2880 - accuracy: 0.7113 - val_loss: 0.2980 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 4/15
47/49 [===========================>..] - ETA: 0s - loss: 0.3029 - accuracy: 0.7447
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.3021 - accuracy: 0.7423 - val_loss: 0.2742 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.2484 - accuracy: 0.7732 - val_loss: 0.2403 - val_accuracy: 0.7083 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2160 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 17ms/step - loss: 0.3119 - accuracy: 0.6804 - val_loss: 0.2141 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 9ms/step - loss: 0.3024 - accuracy: 0.7423 - val_loss: 0.2805 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.2792 - accuracy: 0.7320 - val_loss: 0.2225 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
45/49 [==========================>...] - ETA: 0s - loss: 0.2396 - accuracy: 0.7667
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.2399 - accuracy: 0.7732 - val_loss: 0.2751 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.2378 - accuracy: 0.7320 - val_loss: 0.2081 - val_accuracy: 0.8750 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1988 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.2782 - accuracy: 0.8247 - val_loss: 0.2871 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2739 - accuracy: 0.8144 - val_loss: 0.3117 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2058 - accuracy: 0.7938 - val_loss: 0.2763 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2533 - accuracy: 0.8351 - val_loss: 0.2853 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
44/49 [=========================>....] - ETA: 0s - loss: 0.1994 - accuracy: 0.8068
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.2019 - accuracy: 0.8144 - val_loss: 0.3555 - val_accuracy: 0.5417 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1695 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S4G Matrix Model

In [ ]:
# Define the model
model_s4g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s4g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 10ms/step - loss: 0.5137 - accuracy: 0.4271 - val_loss: 0.3854 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 6ms/step - loss: 0.4028 - accuracy: 0.4792 - val_loss: 0.4006 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3904 - accuracy: 0.4792 - val_loss: 0.5870 - val_accuracy: 0.3200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3688 - accuracy: 0.5417 - val_loss: 0.3437 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 5/15
42/48 [=========================>....] - ETA: 0s - loss: 0.3383 - accuracy: 0.5595
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 0s 7ms/step - loss: 0.3502 - accuracy: 0.5312 - val_loss: 0.3825 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 12ms/step - loss: 0.3381 - accuracy: 0.5876 - val_loss: 0.2634 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.3483 - accuracy: 0.6289 - val_loss: 0.2680 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.3141 - accuracy: 0.6082 - val_loss: 0.3496 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 4/15
43/49 [=========================>....] - ETA: 0s - loss: 0.3359 - accuracy: 0.6395
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.3341 - accuracy: 0.6289 - val_loss: 0.3298 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.3028 - accuracy: 0.6907 - val_loss: 0.2355 - val_accuracy: 0.7083 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2698 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 12ms/step - loss: 0.3474 - accuracy: 0.5464 - val_loss: 0.2755 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.3353 - accuracy: 0.5979 - val_loss: 0.3789 - val_accuracy: 0.5417 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.3356 - accuracy: 0.6289 - val_loss: 0.2825 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 4/15
43/49 [=========================>....] - ETA: 0s - loss: 0.3381 - accuracy: 0.6279
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.3367 - accuracy: 0.6289 - val_loss: 0.3036 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.2706 - accuracy: 0.7732 - val_loss: 0.2809 - val_accuracy: 0.7083 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2463 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 32ms/step - loss: 0.3030 - accuracy: 0.6907 - val_loss: 0.3000 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 15ms/step - loss: 0.2638 - accuracy: 0.7732 - val_loss: 0.2544 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 18ms/step - loss: 0.3644 - accuracy: 0.7010 - val_loss: 0.4060 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 15ms/step - loss: 0.2689 - accuracy: 0.7732 - val_loss: 0.2851 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 15ms/step - loss: 0.2355 - accuracy: 0.7629 - val_loss: 0.2637 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 17ms/step - loss: 0.2409 - accuracy: 0.7629 - val_loss: 0.3097 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2255 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 18ms/step - loss: 0.2286 - accuracy: 0.7938 - val_loss: 0.2155 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 13ms/step - loss: 0.1973 - accuracy: 0.7938 - val_loss: 0.1992 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 13ms/step - loss: 0.2081 - accuracy: 0.7732 - val_loss: 0.3105 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 13ms/step - loss: 0.1893 - accuracy: 0.7835 - val_loss: 0.2787 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
48/49 [============================>.] - ETA: 0s - loss: 0.1557 - accuracy: 0.8333
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 19ms/step - loss: 0.1542 - accuracy: 0.8351 - val_loss: 0.3669 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 21ms/step - loss: 0.1504 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S5G Matrix Model

In [ ]:
# Define the model
model_s5g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s5g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 14ms/step - loss: 0.5232 - accuracy: 0.4375 - val_loss: 0.6237 - val_accuracy: 0.4800 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 7ms/step - loss: 0.4050 - accuracy: 0.5104 - val_loss: 0.3538 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 11ms/step - loss: 0.3788 - accuracy: 0.5208 - val_loss: 0.3506 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3564 - accuracy: 0.5521 - val_loss: 0.3289 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3471 - accuracy: 0.6146 - val_loss: 0.3744 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
45/48 [===========================>..] - ETA: 0s - loss: 0.4365 - accuracy: 0.5556
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 1s 11ms/step - loss:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 18ms/step - loss: 0.4321 - accuracy: 0.4845 - val_loss: 0.2696 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 13ms/step - loss: 0.3277 - accuracy: 0.6907 - val_loss: 0.2825 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 16ms/step - loss: 0.3372 - accuracy: 0.5876 - val_loss: 0.2928 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 4/15
44/49 [=========================>....] - ETA: 0s - loss: 0.3436 - accuracy: 0.6477
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 15ms/step - loss: 0.3517 - accuracy: 0.6289 - val_loss: 0.3043 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 15ms/step - loss: 0.2922 - accuracy: 0.7216 - val_loss: 0.2639 - val_accuracy: 0.8333 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 13ms/step - loss: 0.266

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 2s 16ms/step - loss: 0.3353 - accuracy: 0.6907 - val_loss: 0.2619 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2959 - accuracy: 0.7010 - val_loss: 0.2520 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.3245 - accuracy: 0.6804 - val_loss: 0.2836 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.3015 - accuracy: 0.7320
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.3015 - accuracy: 0.7320 - val_loss: 0.2487 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2390 - accuracy: 0.8144 - val_loss: 0.2131 - val_accuracy: 0.7917 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 10ms/step - loss: 0.3104 - accuracy: 0.6082 - val_loss: 0.2624 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2796 - accuracy: 0.7216 - val_loss: 0.2662 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.2854 - accuracy: 0.6186 - val_loss: 0.2837 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 4/15
44/49 [=========================>....] - ETA: 0s - loss: 0.2390 - accuracy: 0.7386
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.2544 - accuracy: 0.7423 - val_loss: 0.2723 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2035 - accuracy: 0.8144 - val_loss: 0.2469 - val_accuracy: 0.7917 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2154 -

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 3s 16ms/step - loss: 0.3322 - accuracy: 0.6907 - val_loss: 0.3174 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 16ms/step - loss: 0.2660 - accuracy: 0.7629 - val_loss: 0.3905 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 10ms/step - loss: 0.2650 - accuracy: 0.7423 - val_loss: 0.4375 - val_accuracy: 0.5417 - lr: 0.0010
Epoch 4/15
43/49 [=========================>....] - ETA: 0s - loss: 0.2580 - accuracy: 0.7442
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 14ms/step - loss: 0.2409 - accuracy: 0.7732 - val_loss: 0.3007 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 15ms/step - loss: 0.1931 - accuracy: 0.8351 - val_loss: 0.3259 - val_accuracy: 0.5417 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 14ms/step - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S6G Matrix Model

In [ ]:
# Define the model
model_s6g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s6g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 3s 24ms/step - loss: 0.5449 - accuracy: 0.4271 - val_loss: 0.4329 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 1s 16ms/step - loss: 0.4701 - accuracy: 0.4583 - val_loss: 0.3653 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 16ms/step - loss: 0.4077 - accuracy: 0.5312 - val_loss: 0.5061 - val_accuracy: 0.4800 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 14ms/step - loss: 0.4572 - accuracy: 0.4792 - val_loss: 0.3588 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 1s 19ms/step - loss: 0.3719 - accuracy: 0.5729 - val_loss: 0.3570 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 1s 17ms/step - loss: 0.3670 - accuracy: 0.5521 - val_loss: 0.4274 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 1s 15ms/step - loss: 0.3

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 4s 17ms/step - loss: 0.3713 - accuracy: 0.5979 - val_loss: 0.3815 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.3551 - accuracy: 0.6289 - val_loss: 0.2937 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 12ms/step - loss: 0.3289 - accuracy: 0.6495 - val_loss: 0.2236 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 15ms/step - loss: 0.2801 - accuracy: 0.7526 - val_loss: 0.2746 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 18ms/step - loss: 0.2674 - accuracy: 0.7732 - val_loss: 0.2288 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 21ms/step - loss: 0.2440 - accuracy: 0.7835 - val_loss: 0.6714 - val_accuracy: 0.4583 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 2s 33ms/step - loss: 0.3536 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 2s 13ms/step - loss: 0.3441 - accuracy: 0.6495 - val_loss: 0.2466 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2823 - accuracy: 0.7010 - val_loss: 0.2263 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 13ms/step - loss: 0.2821 - accuracy: 0.7423 - val_loss: 0.2451 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2938 - accuracy: 0.6598 - val_loss: 0.2163 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
46/49 [===========================>..] - ETA: 0s - loss: 0.2178 - accuracy: 0.7935
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 12ms/step - loss: 0.2223 - accuracy: 0.7835 - val_loss: 0.2138 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 29ms/step - loss: 0.2734 - accuracy: 0.7320 - val_loss: 0.2108 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 16ms/step - loss: 0.2705 - accuracy: 0.7113 - val_loss: 0.2726 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 19ms/step - loss: 0.2646 - accuracy: 0.7423 - val_loss: 0.2159 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 18ms/step - loss: 0.2589 - accuracy: 0.7216 - val_loss: 0.2400 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 5/15
48/49 [============================>.] - ETA: 0s - loss: 0.2400 - accuracy: 0.7917
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 16ms/step - loss: 0.2391 - accuracy: 0.7938 - val_loss: 0.2734 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 15ms/step - loss: 0.1937 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 19ms/step - loss: 0.2827 - accuracy: 0.7010 - val_loss: 0.2802 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2244 - accuracy: 0.8041 - val_loss: 0.2670 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2057 - accuracy: 0.7938 - val_loss: 0.2884 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 4/15
46/49 [===========================>..] - ETA: 0s - loss: 0.2203 - accuracy: 0.7717
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.2184 - accuracy: 0.7732 - val_loss: 0.2715 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1733 - accuracy: 0.8454 - val_loss: 0.2437 - val_accuracy: 0.6250 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 12ms/step - loss: 0.175

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Standardizing SkipGram

In [ ]:
# standardize the sxgbg_matrix
import numpy as np

def calculate_sxgbg_stan_features(evolutionary_profile, X):
    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1, L - X):
                sxgbg_value += evolutionary_profile[l - 1, i] * evolutionary_profile[l + X, j]

            sxgbg_matrix[i, j] = sxgbg_value

    # Standardize the sxgbg_matrix (z-score normalization)
    mean = np.mean(sxgbg_matrix)
    std = np.std(sxgbg_matrix)

    if std != 0:
        sxgbg_matrix = (sxgbg_matrix - mean) / std
    else:
        sxgbg_matrix = np.zeros_like(sxgbg_matrix)  # Handle the case of zero standard deviation

    return sxgbg_matrix

## S0G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s0g_stan_matrix = []

for i in df['hmm_matrix']:
    s0g_mat = calculate_sxgbg_stan_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_stan_matrix.append(s0g_arr)

df['s0g_stan_matrix'] = s0g_stan_matrix

## S1G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s1g_stan_matrix = []

for i in df['hmm_matrix']:
    s1g_mat = calculate_sxgbg_stan_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_stan_matrix.append(s1g_arr)

df['s1g_stan_matrix'] = s1g_stan_matrix

## S2G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s2g_stan_matrix = []

for i in df['hmm_matrix']:
    s2g_mat = calculate_sxgbg_stan_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_stan_matrix.append(s2g_arr)

df['s2g_stan_matrix'] = s2g_stan_matrix

## S3G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s3g_stan_matrix = []

for i in df['hmm_matrix']:
    s3g_mat = calculate_sxgbg_stan_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_stan_matrix.append(s3g_arr)

df['s3g_stan_matrix'] = s3g_stan_matrix

## S4G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s4g_stan_matrix = []

for i in df['hmm_matrix']:
    s4g_mat = calculate_sxgbg_stan_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_stan_matrix.append(s4g_arr)

df['s4g_stan_matrix'] = s4g_stan_matrix

## S5G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s5g_stan_matrix = []

for i in df['hmm_matrix']:
    s5g_mat = calculate_sxgbg_stan_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_stan_matrix.append(s5g_arr)

df['s5g_stan_matrix'] = s5g_stan_matrix

## S6G Stan

In [ ]:
import numpy as np

# Create an empty list to store hmm_arr for each PDBid
s6g_stan_matrix = []

for i in df['hmm_matrix']:
    s6g_mat = calculate_sxgbg_stan_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_stan_matrix.append(s6g_arr)

df['s6g_stan_matrix'] = s6g_stan_matrix

## S0G Stan Matrix Model

In [ ]:
# Define the model
model_s0g_stan_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_stan_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s0g_stan_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_stan_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 16ms/step - loss: 0.4230 - accuracy: 0.5000 - val_loss: 0.3185 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 7ms/step - loss: 0.3059 - accuracy: 0.6562 - val_loss: 0.3600 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 6ms/step - loss: 0.2422 - accuracy: 0.7604 - val_loss: 0.2967 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.1845 - accuracy: 0.7917 - val_loss: 0.3113 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1547 - accuracy: 0.8646 - val_loss: 0.3077 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1482 - accuracy: 0.8646 - val_loss: 0.2993 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1344 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 10ms/step - loss: 0.2180 - accuracy: 0.8041 - val_loss: 0.1484 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1811 - accuracy: 0.8041 - val_loss: 0.1337 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1240 - accuracy: 0.8763 - val_loss: 0.1104 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0898 - accuracy: 0.8763 - val_loss: 0.1103 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1324 - accuracy: 0.8660 - val_loss: 0.1625 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 6/15
47/49 [===========================>..] - ETA: 0s - loss: 0.1027 - accuracy: 0.9149
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.1009 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1925 - accuracy: 0.8247 - val_loss: 0.0758 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1027 - accuracy: 0.9072 - val_loss: 0.0503 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1137 - accuracy: 0.9175 - val_loss: 0.1326 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1043 - accuracy: 0.9175 - val_loss: 0.1577 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
45/49 [==========================>...] - ETA: 0s - loss: 0.0806 - accuracy: 0.9000
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0835 - accuracy: 0.9072 - val_loss: 0.0845 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0751 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 12ms/step - loss: 0.1292 - accuracy: 0.8763 - val_loss: 0.1442 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 10ms/step - loss: 0.0995 - accuracy: 0.8866 - val_loss: 0.1201 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0570 - accuracy: 0.9175 - val_loss: 0.0847 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0501 - accuracy: 0.9588 - val_loss: 0.0845 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0497 - accuracy: 0.9072 - val_loss: 0.0448 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - ETA: 0s - loss: 0.0770 - accuracy: 0.9278
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 10ms/step - loss: 0.0770 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.0932 - accuracy: 0.9175 - val_loss: 0.0523 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0679 - accuracy: 0.9485 - val_loss: 0.0765 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0336 - accuracy: 0.9381 - val_loss: 0.0619 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0503 - accuracy: 0.9485 - val_loss: 0.2951 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/15
48/49 [============================>.] - ETA: 0s - loss: 0.1210 - accuracy: 0.8958
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1197 - accuracy: 0.8969 - val_loss: 0.1835 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0708 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S1G Stan Matrix Model

In [ ]:
# Define the model
model_s1g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s1g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 9ms/step - loss: 0.5220 - accuracy: 0.3854 - val_loss: 0.3397 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3287 - accuracy: 0.5938 - val_loss: 0.3133 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2581 - accuracy: 0.7292 - val_loss: 0.3403 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 8ms/step - loss: 0.2026 - accuracy: 0.8542 - val_loss: 0.3216 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1870 - accuracy: 0.8125 - val_loss: 0.3049 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
46/48 [===========================>..] - ETA: 0s - loss: 0.1770 - accuracy: 0.8696
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 0s 7ms/step - loss: 0.17

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.3202 - accuracy: 0.6907 - val_loss: 0.1989 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2476 - accuracy: 0.7732 - val_loss: 0.3064 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2455 - accuracy: 0.7320 - val_loss: 0.2423 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
42/49 [========================>.....] - ETA: 0s - loss: 0.1662 - accuracy: 0.8690
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1642 - accuracy: 0.8763 - val_loss: 0.1752 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1144 - accuracy: 0.8866 - val_loss: 0.2827 - val_accuracy: 0.7083 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0990 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.2946 - accuracy: 0.7320 - val_loss: 0.2062 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2036 - accuracy: 0.8351 - val_loss: 0.1782 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1613 - accuracy: 0.8454 - val_loss: 0.2287 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
42/49 [========================>.....] - ETA: 0s - loss: 0.1312 - accuracy: 0.8810
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1250 - accuracy: 0.8866 - val_loss: 0.2404 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0975 - accuracy: 0.9691 - val_loss: 0.1441 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0759 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.2384 - accuracy: 0.7835 - val_loss: 0.1604 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 10ms/step - loss: 0.1877 - accuracy: 0.8247 - val_loss: 0.1552 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.1560 - accuracy: 0.8454 - val_loss: 0.1548 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
45/49 [==========================>...] - ETA: 0s - loss: 0.1529 - accuracy: 0.8667
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1490 - accuracy: 0.8763 - val_loss: 0.1568 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0886 - accuracy: 0.9485 - val_loss: 0.1399 - val_accuracy: 0.8333 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 8ms/step - loss: 0.0692 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 10ms/step - loss: 0.1922 - accuracy: 0.8454 - val_loss: 0.2226 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1440 - accuracy: 0.8866 - val_loss: 0.2528 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1052 - accuracy: 0.8866 - val_loss: 0.2283 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1194 - accuracy: 0.8866 - val_loss: 0.6112 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 5/15
43/49 [=========================>....] - ETA: 0s - loss: 0.1214 - accuracy: 0.8953
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.1203 - accuracy: 0.8969 - val_loss: 0.2502 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0508 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S2G Stan Matrix Model

In [ ]:
# Define the model
model_s2g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s2g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 11ms/step - loss: 0.4550 - accuracy: 0.4479 - val_loss: 0.3827 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3818 - accuracy: 0.5208 - val_loss: 0.3288 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 7ms/step - loss: 0.3197 - accuracy: 0.6562 - val_loss: 0.3067 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.2781 - accuracy: 0.6771 - val_loss: 0.3524 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2209 - accuracy: 0.8021 - val_loss: 0.3348 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 6ms/step - loss: 0.1707 - accuracy: 0.8438 - val_loss: 0.2922 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 6ms/step - loss: 0.1558 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 14ms/step - loss: 0.1606 - accuracy: 0.8660 - val_loss: 0.0622 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1688 - accuracy: 0.8557 - val_loss: 0.1149 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1271 - accuracy: 0.8660 - val_loss: 0.1285 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.1123 - accuracy: 0.8969
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1123 - accuracy: 0.8969 - val_loss: 0.0639 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0704 - accuracy: 0.9381 - val_loss: 0.0468 - val_accuracy: 1.0000 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0588 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1101 - accuracy: 0.8969 - val_loss: 0.0342 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1043 - accuracy: 0.8969 - val_loss: 0.0999 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0873 - accuracy: 0.9072 - val_loss: 0.0293 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.0714 - accuracy: 0.9485
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0714 - accuracy: 0.9485 - val_loss: 0.0599 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0295 - accuracy: 0.9691 - val_loss: 0.0493 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0182 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1182 - accuracy: 0.8454 - val_loss: 0.0931 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0765 - accuracy: 0.9278 - val_loss: 0.0324 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.0573 - accuracy: 0.9072 - val_loss: 0.0333 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 10ms/step - loss: 0.0263 - accuracy: 0.9588 - val_loss: 0.0435 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0365 - accuracy: 0.9381 - val_loss: 0.0591 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
47/49 [===========================>..] - ETA: 0s - loss: 0.0189 - accuracy: 0.9574
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.0183 - acc

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 10ms/step - loss: 0.0677 - accuracy: 0.9381 - val_loss: 0.0430 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0531 - accuracy: 0.9175 - val_loss: 0.0467 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0248 - accuracy: 0.9485 - val_loss: 0.0759 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0324 - accuracy: 0.9485 - val_loss: 0.0300 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
46/49 [===========================>..] - ETA: 0s - loss: 0.0113 - accuracy: 0.9565
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0121 - accuracy: 0.9588 - val_loss: 0.1323 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0207 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S2G Stan LSTM Matrix Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Define the model
model_lstmsn = keras.Sequential([
    keras.layers.Input(shape=(400,1)),
    keras.layers.LSTM(256, return_sequences=True, kernel_initializer = 'he_normal'),
    keras.layers.LSTM(128, return_sequences=True, kernel_initializer = 'he_normal'),  # Another LSTM layer with 64 units
    keras.layers.Flatten(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu', kernel_initializer = 'he_normal'),     # Fully connected layer
    keras.layers.Dense(5, activation='sigmoid')    # Output layer with sigmoid activation for multi-label classification
])

train_features = np.array(df['s2g_stan_matrix'].tolist())
train_features = train_features.reshape(-1, 400,1)

train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_lstmsn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=7, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=4, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_lstmsn.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_lstmsn.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy+avg_precision+avg_recall+f1+avg_acc_norm++avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 11s 45ms/step - loss: 0.7012 - accuracy: 0.4271 - val_loss: 0.4999 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 1s 27ms/step - loss: 0.3479 - accuracy: 0.6250 - val_loss: 0.3992 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 27ms/step - loss: 0.2966 - accuracy: 0.7708 - val_loss: 0.3331 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 27ms/step - loss: 0.2296 - accuracy: 0.7083 - val_loss: 0.2876 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 1s 27ms/step - loss: 0.1659 - accuracy: 0.8125 - val_loss: 0.2829 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 2s 32ms/step - loss: 0.1177 - accuracy: 0.9167 - val_loss: 0.3044 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 2s 41ms/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 11s 65ms/step - loss: 0.2913 - accuracy: 0.6804 - val_loss: 0.1603 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 3s 55ms/step - loss: 0.1924 - accuracy: 0.8041 - val_loss: 0.1456 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 5s 95ms/step - loss: 0.1587 - accuracy: 0.8247 - val_loss: 0.1793 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 3s 52ms/step - loss: 0.0849 - accuracy: 0.8866 - val_loss: 0.2377 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 2s 39ms/step - loss: 0.1158 - accuracy: 0.9072 - val_loss: 0.2945 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 2s 41ms/step - loss: 0.0598 - accuracy: 0.9588 - val_loss: 0.1839 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 7/15
47/49 [===========================>..] - ETA: 0s - loss: 0.0108 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 11s 94ms/step - loss: 0.1850 - accuracy: 0.8454 - val_loss: 0.0558 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 2s 40ms/step - loss: 0.1354 - accuracy: 0.8969 - val_loss: 0.0917 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 2s 42ms/step - loss: 0.0711 - accuracy: 0.9278 - val_loss: 0.0830 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 2s 41ms/step - loss: 0.0376 - accuracy: 0.9691 - val_loss: 0.1663 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - ETA: 0s - loss: 0.0356 - accuracy: 0.9588
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 2s 42ms/step - loss: 0.0356 - accuracy: 0.9588 - val_loss: 0.1076 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 2s 45ms/step - loss: 0.0094 -

...

49/49 [==============================] - 6s 46ms/step - loss: 0.1668 - accuracy: 0.8557 - val_loss: 0.0511 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 2s 33ms/step - loss: 0.0829 - accuracy: 0.8969 - val_loss: 0.1330 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 2s 40ms/step - loss: 0.0718 - accuracy: 0.9175 - val_loss: 0.1582 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 2s 36ms/step - loss: 0.0346 - accuracy: 0.9175 - val_loss: 0.1257 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 28ms/step - loss: 0.0114 - accuracy: 0.9588 - val_loss: 0.1155 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
48/49 [============================>.] - ETA: 0s - loss: 0.0180 - accuracy: 0.9375
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 2s 32ms/step - loss: 0.0178 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 6s 47ms/step - loss: 0.1400 - accuracy: 0.8866 - val_loss: 0.0605 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 27ms/step - loss: 0.0491 - accuracy: 0.9175 - val_loss: 0.0764 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 28ms/step - loss: 0.0232 - accuracy: 0.9485 - val_loss: 0.1064 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 2s 50ms/step - loss: 0.0094 - accuracy: 0.9691 - val_loss: 0.0858 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - ETA: 0s - loss: 0.0025 - accuracy: 0.9588
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 2s 38ms/step - loss: 0.0025 - accuracy: 0.9588 - val_loss: 0.0758 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 27ms/step - loss: 4.4919e-0

1/1 [==============================] - 1s 624ms/step
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 1] [0 0 0 1 1]
1.0
[0 1 0 0 0] [0 1 0 0 1]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 1] [1 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 1 0 0 0] [0 1 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
Average Accuracy:  0.8356666666666666
Accuracy:  0.9541333333333334
Average Normalized Accuracy:  0.8523333333333334
Average Precision:  0.9039813519813521
Average Recall:  0.8593846153846154
F1 score: 0.8811190426316404
Grand Mean: 0.8811030572218234


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S2G Stan XGBoost Matrix Model

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Define the XGBoost classifier
model_xgb = xgb.XGBClassifier()

# Load your data and pr0.0.eprocess it
train_features = np.array(df['s2g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Fit the XGBoost model to the training data
    model_xgb.fit(X_train, y_train)

    # Make predictions on the current test sample
    y_pred = model_xgb.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy+avg_precision+avg_recall+f1+avg_acc_norm++avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [0 0 0 1 0]
0.6
[0 0 0 1 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 1 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 1 0]
0.6
[0 1 0 0 0] [0 0 0 1 1]
0.4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1 0 0 0 1] [1 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [0 0 0 1 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 1 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 1]
0.6
[0 0 0 0 1] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 1] [1 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [1 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1 0 0 0 0] [1 0 0 0 1]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 1] [1 0 0 0 0]
0.4
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 1] [1 0 0 0 0]
0.8
[1 0 0 1 0] [0 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 1] [1 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 0 0]
0.8


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 1] [0 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 1]
0.8
[1 0 0 0 0] [0 1 0 0 0]
0.6
[1 0 0 0 0] [0 0 0 0 1]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[1 0 0 0 1] [1 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 1]
0.8
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 1 1]
0.8
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [1 0 0 0 1]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [1 0 0 0 1]
0.8
[0 0 0 0 1] [0 0 0 1 0]
0.6
[0 0 0 0 1] [1 0 0 1 0]
0.4
[0 0 0 1 1] [0 0 0 1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S2G Stan KNN Matrix Model

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold

# Define the k-NN classifier
model_knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed

# Load your data and preprocess it
train_features = np.array(df['s2g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Fit the k-NN model to the training data
    model_knn.fit(X_train, y_train)

    # Make predictions on the current test sample
    y_pred = model_knn.predict(X_test)

    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy+avg_precision+avg_recall+f1+avg_acc_norm++avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall

[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 1 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 1]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 1]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 1]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 1 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 1 0]
0.6
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 1 0]
0.6
[0 1 0 0 0] [0 0 0 0 1]
0.6
[1 0 0 0 1] [1 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 1 0]
0.6
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 1 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [0 0 0 1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall

[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 1 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 1] [1 0 0 0 1]
0.6
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 0 1] [0 0 0 1 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 1] [0 0 0 0 0]
0.6
[1 0 0 1 0] [0 0 0 0 1]
0.4
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [0 0 0 0 1]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 1] [1 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 1] [1 0 0 0 0]
0.4
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [0 0 0 1 0]
0.6
[1 0 0 0 0] [0 0 0 0 0]
0.8
[1 0 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S3G Stan Matrix Model

In [ ]:
# Define the model
model_s3g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s3g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 13ms/step - loss: 0.4608 - accuracy: 0.4792 - val_loss: 0.3471 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 8ms/step - loss: 0.3175 - accuracy: 0.5729 - val_loss: 0.3641 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 8ms/step - loss: 0.2904 - accuracy: 0.6875 - val_loss: 0.3238 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2421 - accuracy: 0.7396 - val_loss: 0.2984 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1880 - accuracy: 0.8438 - val_loss: 0.3423 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1686 - accuracy: 0.8854 - val_loss: 0.3535 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1572 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 16ms/step - loss: 0.2874 - accuracy: 0.7629 - val_loss: 0.1724 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 12ms/step - loss: 0.1979 - accuracy: 0.8351 - val_loss: 0.2647 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 10ms/step - loss: 0.1699 - accuracy: 0.7938 - val_loss: 0.1499 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
42/49 [========================>.....] - ETA: 0s - loss: 0.1706 - accuracy: 0.8452
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1630 - accuracy: 0.8454 - val_loss: 0.1507 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1290 - accuracy: 0.8866 - val_loss: 0.1248 - val_accuracy: 0.8750 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0898 -

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 12ms/step - loss: 0.1745 - accuracy: 0.8763 - val_loss: 0.0519 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1053 - accuracy: 0.9072 - val_loss: 0.0495 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1287 - accuracy: 0.8557 - val_loss: 0.0684 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 8ms/step - loss: 0.0521 - accuracy: 0.9588 - val_loss: 0.0943 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - ETA: 0s - loss: 0.0581 - accuracy: 0.9175
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 9ms/step - loss: 0.0581 - accuracy: 0.9175 - val_loss: 0.1446 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0410 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.1124 - accuracy: 0.9175 - val_loss: 0.0533 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0973 - accuracy: 0.9072 - val_loss: 0.0459 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1087 - accuracy: 0.8969 - val_loss: 0.1105 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 12ms/step - loss: 0.0601 - accuracy: 0.9278 - val_loss: 0.0506 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
43/49 [=========================>....] - ETA: 0s - loss: 0.0462 - accuracy: 0.9651
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.0467 - accuracy: 0.9485 - val_loss: 0.0912 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 10ms/step - loss: 0.0358 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 10ms/step - loss: 0.0972 - accuracy: 0.9175 - val_loss: 0.0840 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0699 - accuracy: 0.9175 - val_loss: 0.0551 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0463 - accuracy: 0.9588 - val_loss: 0.1989 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0896 - accuracy: 0.8866 - val_loss: 0.1078 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
42/49 [========================>.....] - ETA: 0s - loss: 0.0675 - accuracy: 0.9048
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0696 - accuracy: 0.9175 - val_loss: 0.1221 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0391 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S4G Stan Matrix Model

In [ ]:
# Define the model
model_s4g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s4g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 12ms/step - loss: 0.4802 - accuracy: 0.4167 - val_loss: 0.3312 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3187 - accuracy: 0.5938 - val_loss: 0.3120 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2632 - accuracy: 0.6771 - val_loss: 0.2730 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.2043 - accuracy: 0.7917 - val_loss: 0.2978 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1624 - accuracy: 0.8542 - val_loss: 0.3724 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 7ms/step - loss: 0.1524 - accuracy: 0.8125 - val_loss: 0.2497 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 8ms/step - loss: 0.1338 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.1711 - accuracy: 0.8557 - val_loss: 0.1061 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1338 - accuracy: 0.8351 - val_loss: 0.1135 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.1453 - accuracy: 0.8763 - val_loss: 0.1578 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0950 - accuracy: 0.8763 - val_loss: 0.0803 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0811 - accuracy: 0.8969 - val_loss: 0.3480 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1302 - accuracy: 0.8866 - val_loss: 0.0884 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 7/15
40/49 [=======================>......] - ETA: 0s - loss: 0.0765 - accuracy: 0.91

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.1335 - accuracy: 0.8660 - val_loss: 0.0846 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1072 - accuracy: 0.8969 - val_loss: 0.0516 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0760 - accuracy: 0.9485 - val_loss: 0.0855 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0405 - accuracy: 0.9691 - val_loss: 0.0850 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
44/49 [=========================>....] - ETA: 0s - loss: 0.0582 - accuracy: 0.9205
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0542 - accuracy: 0.9278 - val_loss: 0.0949 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0198 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 10ms/step - loss: 0.1019 - accuracy: 0.9381 - val_loss: 0.0700 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0767 - accuracy: 0.9175 - val_loss: 0.0447 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0523 - accuracy: 0.9485 - val_loss: 0.0344 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
41/49 [========================>.....] - ETA: 0s - loss: 0.0536 - accuracy: 0.9390
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0498 - accuracy: 0.9381 - val_loss: 0.0485 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0320 - accuracy: 0.9278 - val_loss: 0.0662 - val_accuracy: 1.0000 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0134 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 10ms/step - loss: 0.0942 - accuracy: 0.9072 - val_loss: 0.0561 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0520 - accuracy: 0.9381 - val_loss: 0.0343 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0215 - accuracy: 0.9485 - val_loss: 0.0278 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
39/49 [======================>.......] - ETA: 0s - loss: 0.0197 - accuracy: 0.9487
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0174 - accuracy: 0.9588 - val_loss: 0.0230 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0120 - accuracy: 0.9588 - val_loss: 0.0289 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0051 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S5G Stan Matrix Model

In [ ]:
# Define the model
model_s5g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s5g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 10ms/step - loss: 0.4700 - accuracy: 0.4271 - val_loss: 0.3403 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 7ms/step - loss: 0.3647 - accuracy: 0.5208 - val_loss: 0.3062 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 7ms/step - loss: 0.3575 - accuracy: 0.5625 - val_loss: 0.3214 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2933 - accuracy: 0.6562 - val_loss: 0.3075 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 5/15
46/48 [===========================>..] - ETA: 0s - loss: 0.2477 - accuracy: 0.7500
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 0s 6ms/step - loss: 0.2460 - accuracy: 0.7604 - val_loss: 0.3108 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 6ms/step - loss: 0.2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.2625 - accuracy: 0.7113 - val_loss: 0.1376 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2181 - accuracy: 0.7938 - val_loss: 0.2352 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 10ms/step - loss: 0.1822 - accuracy: 0.8454 - val_loss: 0.1502 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.1569 - accuracy: 0.8351
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.1569 - accuracy: 0.8351 - val_loss: 0.1174 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1071 - accuracy: 0.9381 - val_loss: 0.1617 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1038

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1615 - accuracy: 0.8351 - val_loss: 0.0873 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1158 - accuracy: 0.8969 - val_loss: 0.0495 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1267 - accuracy: 0.9072 - val_loss: 0.0723 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1150 - accuracy: 0.8866 - val_loss: 0.1088 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
44/49 [=========================>....] - ETA: 0s - loss: 0.1007 - accuracy: 0.9205
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0938 - accuracy: 0.9278 - val_loss: 0.0641 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0489 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 14ms/step - loss: 0.1473 - accuracy: 0.8866 - val_loss: 0.0386 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0917 - accuracy: 0.9175 - val_loss: 0.0890 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0788 - accuracy: 0.8763 - val_loss: 0.1176 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
45/49 [==========================>...] - ETA: 0s - loss: 0.1308 - accuracy: 0.8556
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 10ms/step - loss: 0.1267 - accuracy: 0.8454 - val_loss: 0.0727 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0486 - accuracy: 0.9588 - val_loss: 0.0606 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 12ms/step - loss: 0.036

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1232 - accuracy: 0.9072 - val_loss: 0.1084 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1297 - accuracy: 0.8557 - val_loss: 0.2934 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0940 - accuracy: 0.9175 - val_loss: 0.0789 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0938 - accuracy: 0.9485 - val_loss: 0.1275 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0716 - accuracy: 0.9175 - val_loss: 0.1304 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
42/49 [========================>.....] - ETA: 0s - loss: 0.0511 - accuracy: 0.9762
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0487 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S6G Stan Matrix Model

In [ ]:
# Define the model
model_s6g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s6g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 11ms/step - loss: 0.5019 - accuracy: 0.3958 - val_loss: 0.3532 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3496 - accuracy: 0.6146 - val_loss: 0.3303 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2738 - accuracy: 0.7292 - val_loss: 0.3168 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2018 - accuracy: 0.8125 - val_loss: 0.3120 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 10ms/step - loss: 0.1647 - accuracy: 0.8750 - val_loss: 0.3409 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 10ms/step - loss: 0.1441 - accuracy: 0.8854 - val_loss: 0.3137 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 1s 11ms/step - loss: 0.1239

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 9ms/step - loss: 0.1976 - accuracy: 0.8351 - val_loss: 0.1287 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1486 - accuracy: 0.8557 - val_loss: 0.1545 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1570 - accuracy: 0.8557 - val_loss: 0.1160 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
44/49 [=========================>....] - ETA: 0s - loss: 0.1151 - accuracy: 0.9091
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1124 - accuracy: 0.9072 - val_loss: 0.0980 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0663 - accuracy: 0.9381 - val_loss: 0.1216 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0546 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.1844 - accuracy: 0.8247 - val_loss: 0.0993 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1328 - accuracy: 0.9175 - val_loss: 0.0903 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1075 - accuracy: 0.9175 - val_loss: 0.2111 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
42/49 [========================>.....] - ETA: 0s - loss: 0.0967 - accuracy: 0.8690
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1013 - accuracy: 0.8660 - val_loss: 0.1124 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0720 - accuracy: 0.9278 - val_loss: 0.1000 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0430 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1848 - accuracy: 0.8557 - val_loss: 0.0965 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1278 - accuracy: 0.8969 - val_loss: 0.0825 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0917 - accuracy: 0.8866 - val_loss: 0.2369 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
41/49 [========================>.....] - ETA: 0s - loss: 0.0883 - accuracy: 0.9146
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1088 - accuracy: 0.9072 - val_loss: 0.1522 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0580 - accuracy: 0.9485 - val_loss: 0.0733 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0370 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 18ms/step - loss: 0.1365 - accuracy: 0.8969 - val_loss: 0.1891 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 12ms/step - loss: 0.1063 - accuracy: 0.8866 - val_loss: 0.1554 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 12ms/step - loss: 0.1059 - accuracy: 0.9072 - val_loss: 0.1087 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0933 - accuracy: 0.9381 - val_loss: 0.1557 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0537 - accuracy: 0.9381 - val_loss: 0.1493 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
40/49 [=======================>......] - ETA: 0s - loss: 0.0395 - accuracy: 0.9500
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0460 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
